In [1]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nii_gz_path = 'C:/Users/panag/Desktop/Test/mgh_1001/diff/preproc/mri/diff_preproc.nii.gz'

In [3]:
nii_img  = nib.load(nii_gz_path)

In [4]:
nii_data = nii_img.get_fdata()

In [5]:
num_x_slices, num_y_slices, num_z_sclices, num_directions = nii_data.shape

Read vectors and plot 3d